# Classification de données non linéaires avec une couche cachée

Bonjour et bienvenue à ce workshop dans lequel nous allons construire notre premier réseau de neurones profond (deep learning), contenant une couche cachée. Vous remarquerez une grosse différence entre ce modèle et celui que vous avez implémenté avec la régression logistique.


**Ce que vous allez apprendre:**
- Implémenter un réseau de neurones avec une couche cachée pour une classification à 2 classes
- Utiliser les neurones avec une fonction d'activation non linéaire, comme la tanh
- Calculer la loss cross-entropy
- Implémenter les forward et backward propagations

## 1 - Packages ##

Commençons par importer les packages dont nous aurons besoin:
- [numpy](www.numpy.org) est le package fondamental pour le calcul scientifique en Python.
- [sklearn](http://scikit-learn.org/stable/) dispose de nombreuses fonctions pour la construction et l'analyse des données. 
- [matplotlib](http://matplotlib.org) est une librairie pour afficher des graphes, tableaux, etc. en Python.
- tests fournit quelques exemples tests pour vérifier vos fonctions
- utils fournit de nombreuses fonctions utiles dont nous avons besoin pour ce notebook

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tests import *
import sklearn
import sklearn.datasets
import sklearn.linear_model
from utils import plot_decision_boundary, sigmoid, load_planar_dataset, load_extra_datasets

%matplotlib inline

np.random.seed(1) # seed pour comparer avec les valeurs attendues, ne pas modifier !!

## 2 - Dataset ##

Tout d'abord, récupérons le jeu de données. Le code suivant va initialiser un jeu de données "fleur" à 2 classes dans les  variables `X` et `Y`.

In [ ]:
X, Y = load_planar_dataset()

Visualisez le jeu de données avec matplotlib. Les données ressemblent à une "fleur" avec des points rouges (label y = 0) et bleus (y = 1). Votre but est de construire un modèle qui classeront correctement ces données.

In [ ]:
c = []
for i in range(Y.shape[1]):
    if i < Y.shape[1] / 2: c.append('red')
    else: c.append('blue')
plt.scatter(X[0, :], X[1, :], c=c, s=40, cmap=plt.cm.Spectral);

Vous avez:
    - un numpy-array (matrix) X qui contient vos features (x1, x2)
    - un numpy-array (vecteur) Y qui contient vos labels (rouge:0, bleu:1).

Analysons davantage nos données.

**Exercice**: Combien d'exemples de train avez vous ? De plus, quels sont les shapes des variables `X` et `Y`? 

**Indice**: Comment récupérez vous le shape d'un numpy array ? [(help)](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.shape.html)

In [ ]:
### Début du code ### (≈ 3 lignes de code)
shape_X = None
shape_Y = None
### Fin du code ###

print ('Le shape de X est: ' + str(shape_X))
print ('Le shape de Y est: ' + str(shape_Y))

**Résultat attendu**:
       
<table style="width:20%">
  
  <tr>
    <td>shape de X</td>
    <td> (2, 400) </td> 
  </tr>
  
  <tr>
    <td>shape de Y</td>
    <td>(1, 400) </td> 
  </tr>

  </tr>
  
</table>

## 3 - Simple régression logistiaue

Avant de construire un réseau de neurones complet, visualisons les performances de la régression logistique à ce problème. Vous pouvez utilisez les fonctions de sklearn pour le faire. Exécutez la cellule suivant pour entraîner une régression logistique sur notre jeu de données. 

In [ ]:
# Entrainement du classifier régression logistique
clf = sklearn.linear_model.LogisticRegressionCV(cv=5);
clf.fit(X.T, Y.T.ravel());

Vous pouvez maintenant plotter le decision boundary (limite de décision) de ce modèle. Exécutez le code suivant. 

**Attention:** Suite à des problèmes de versions matplotlib, il se peut que vous ne voyez pas les points initiaux. L'important est de voir comment la régression logistique a apporté sa solution au problème. Appelez un assistant si vous ne comprenez pas.

In [ ]:
# Plot de la decision boundary pour la régression logistique
try:
    plot_decision_boundary(lambda x: clf.predict(x), X, Y)
except:
    print("Suite à un problème de version matplotlib, vous ne voyez pas les points initiaux. Reprenez le graphique précédent et imaginez les points comme étant plot.")
    pass
plt.title("Régression logistique")

# Print l'accuracy
LR_predictions = clf.predict(X.T)
print ('Accuracy de la régression logistique: %d ' % float((np.dot(Y,LR_predictions) + np.dot(1-Y,1-LR_predictions))/float(Y.size)*100) +
       '% ' + "de points labellisés corrects")

**Résultat attendu**:

<table style="width:20%">
  <tr>
    <td>Accuracy</td>
    <td> 47% </td> 
  </tr>
  
</table>


**Interpretation**: Ce jeu de données n'est pas linéairement séparable, donc la logistique régression ne fonctionne pas ici. Heureusement, un réseau de neurones profond pourra résoudre notre problème. C'est parti ! 

## 4 - Réseau de neurones profond

La régression logistique ne fonctionne pas sur notre jeu de données. Vous allez maintenant entraîner un réseau de neurones profond avec une couche cachée.

**Voici notre modèle**:
<img src="images/classification_kiank.png" style="width:600px;height:300px;">

**Mathématiquement**:

Pour un exemple $x^{(i)}$:
$$z^{[1] (i)} =  W^{[1]} x^{(i)} + b^{[1]}\tag{1}$$ 
$$a^{[1] (i)} = \tanh(z^{[1] (i)})\tag{2}$$
$$z^{[2] (i)} = W^{[2]} a^{[1] (i)} + b^{[2]}\tag{3}$$
$$\hat{y}^{(i)} = a^{[2] (i)} = \sigma(z^{ [2] (i)})\tag{4}$$
$$y^{(i)}_{prediction} = \begin{cases} 1 & \mbox{si } a^{[2](i)} > 0.5 \\ 0 & \mbox{sinon } \end{cases}\tag{5}$$

A partir des prédictions calculées, vous pourrez calculer le coût grâce à la formule suivante:
$$J = - \frac{1}{m} \sum\limits_{i = 0}^{m} \large\left(\small y^{(i)}\log\left(a^{[2] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[2] (i)}\right)  \large  \right) \small \tag{6}$$

**Rappel**: La méthodologie générale pour construire un réseau de neurones est la suivante:
    1. Définir la structure du réseau de neurones ( nombre de features en entrée,  nombre de couches cachées, etc). 
    2. Initialiser les paramètres du modèle
    3. Boucle:
        - Implémenter la forward propagation
        - Calcul du coût
        - Implémenter la backward propagation pour avoir les gradients
        - Update des paramètres 

Après avoir créé vos fonctions, vous allez les merger en une unique fonction qu'on nommera `nn_model()`. 
Quand ce modèle sera entraîné, vous pourrez réaliser des prédictions sur des nouvelles données.

### 4.1 - Définir la structure du réseau de neurones ####

**Exercice**: Définissez 3 variables:
    - n_x: size de la couche d'entrée
    - n_h: size de la couche cachée (définissez la à 4) 
    - n_y: size de la couche de sortie

**Indice**: Utilisez les shapes de X et Y pour trouver n_x et n_y. Hardcodez directement la couche cachée.

In [ ]:
def layer_sizes(X, Y):
    """
    Arguments:
    X -- jeu de données en entrée de shape (size en entrée, nombre d'exemples)
    Y -- labels de shape (size en sortie, nombre d'exemples)
    """
    ### Début du code ### (≈ 3 lignes de code)
    n_x = None # size de la couche d'entrée
    n_h = None
    n_y = None # size de la couche de sortie
    ### Fin du code ###
    return (n_x, n_h, n_y)

In [ ]:
X_assess, Y_assess = layer_sizes_test_case()
(n_x, n_h, n_y) = layer_sizes(X_assess, Y_assess)
print("Size de la couche d'entrée: n_x = " + str(n_x))
print("Size de la couche cachée: n_h = " + str(n_h))
print("Size de la couche de sortie: n_y = " + str(n_y))

**Résultat attentu** (il ne s'agit pas du nombres de neurones que nous utiliserons pour notre modèle, juste un test pour vérifier votre fonction).

<table style="width:20%">
  <tr>
    <td>n_x</td>
    <td> 5 </td> 
    </tr>
    <tr>
    <td>n_h</td>
    <td> 4 </td> 
  </tr>
    <tr>
    <td>n_y</td>
    <td> 2 </td> 
  </tr>
  
</table>

### 4.2 -Initialiser les paramètres du modèle ####

**Exercice**: Implementez la function `initialize_parameters()`.

**Instructions**:
- Soyez sûrs que les sizes de vos paramètres sont correctes. Regardez le schéma au-dessus si nécessaire.
- Vous initialiserez les matrices des poids avec des valeurs aléatoires
    - Utilisez: `np.random.randn(a,b) * 0.01` pour initialiser aléatoirement une matrice de shape (a,b).
- Vous initialisez les vectors des biais à 0.
    - Utilisez: `np.zeros((a,b))` pour initialiser une matrice de shape (a,b) avec des zéros.

In [ ]:
def initialize_parameters(n_x, n_h, n_y):
    """
    Argument:
    n_x -- size de la couche d'entrée
    n_h -- size de la couche cachée (définissez la à 4) 
    n_y -- size de la couche de sortie
    
    Return:
    params -- dictionnaire python contenant les paramètres:
                    W1 -- matrice poids de shape (n_h, n_x)
                    b1 -- vecteur biais shape (n_h, 1)
                    W2 -- matrice poids shape (n_y, n_h)
                    b2 -- vecteur biais shape (n_y, 1)
    """
    
    np.random.seed(42) # ne pas modifier le seed, il nous servira à comparer nos résultats
    
    ### Début du code ### (≈ 4 lignes de code)
    W1 = None
    b1 = None
    W2 = None
    b2 = None
    ### Fin du code ###
    
    assert (W1.shape == (n_h, n_x))
    assert (b1.shape == (n_h, 1))
    assert (W2.shape == (n_y, n_h))
    assert (b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [ ]:
n_x, n_h, n_y = initialize_parameters_test_case()

parameters = initialize_parameters(n_x, n_h, n_y)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

**Résultat attendu**:

<table style="width:90%">
  <tr>
    <td>W1</td>
    <td> [[ 0.00496714 -0.00138264]
 [ 0.00647689  0.0152303 ]
 [-0.00234153 -0.00234137]
 [ 0.01579213  0.00767435]] </td> 
  </tr>
  
  <tr>
    <td>b1</td>
    <td> [[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]] </td> 
  </tr>
  
  <tr>
    <td>W2</td>
    <td> [[-0.00469474  0.0054256  -0.00463418 -0.0046573 ]]</td> 
  </tr>
  

  <tr>
    <td>b2</td>
    <td> [[ 0.]] </td> 
  </tr>
  
</table>



### 4.3 -La boucle ####

**Exercice**: Implementez `forward_propagation()`.

**Instructions**:
- Regardez au-dessus les formules mathématiques de votre modèle.
- Vous pouvez utiliser la fonction `sigmoid()`. Nous l'avons défini et importé au début depuis utils.
- Vous pouvez utiliser la fonction `np.tanh()`. C'est une fonction de la lib numpy.
- Les étapes que vous devez implémenter sont les suivantes:
    1. Récuperez chaque paramètre à partir du dictionnaire python "parameters" (qui correspond à l'output de `initialize_parameters()`). 
    2. Implémentez la forward propagation. Calculez $Z^{[1]}, A^{[1]}, Z^{[2]}$ et $A^{[2]}$. Rappel: Z correspond à la pré-activation et A à l'activation.
- Les valeurs dont vous aurez besoin pour la backpropagation sont stockées dans "`cache`". Le dictionnaire `cache` sera un argument de la fonction backpropagation.

In [ ]:
def forward_propagation(X, parameters):
    """
    Argument:
    X -- données en entrée de size (n_x, m)
    parameters -- dictionnaire python contenant vos paramètres (poids et biais)
    
    Return:
    A2 -- L'output de la sigmoïde de la deuxième activation
    cache -- dictionnaire contenant "Z1", "A1", "Z2" et "A2"
    """
    # Récupérez chaque paramètre du dictionnaire "parameters"
    ### Début du code ### (≈ 4 lignes de code)
    W1 = None
    b1 = None
    W2 = None
    b2 = None
    ### Fin du code ###
    
    # Implémentez la forward porpagation pour calculer A2 (probabilités)
    ### Début du code ### (≈ 4 lignes de code)
    Z1 = None
    A1 = None
    Z2 = None
    A2 = None
    ### Fin du code ###
    
    assert(A2.shape == (1, X.shape[1]))
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

In [ ]:
X_assess, parameters = forward_propagation_test_case()
A2, cache = forward_propagation(X_assess, parameters)

# Note: on utilise la moyenne ici pour vérifier que vos résultats matchent avec les notres
print(np.mean(cache['Z1']) ,np.mean(cache['A1']),np.mean(cache['Z2']),np.mean(cache['A2']))

**Résultat attendu**:
<table style="width:50%">
  <tr>
    <td> 0.262818640198 0.091999045227 -1.30766601287 0.212877681719 </td> 
  </tr>
</table>

Maintenant que vous avez calculé $A^{[2]}$ (dans la variable Python "`A2`"), qui contient $a^{[2](i)}$ pour tous les exemples, vous pouvez calculer le coût de la manière suivante:

$$J = - \frac{1}{m} \sum\limits_{i = 0}^{m} \large{(} \small y^{(i)}\log\left(a^{[2] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[2] (i)}\right) \large{)} \small\tag{13}$$

**Exercice**: Implementez `compute_cost()` pour calculer la valeur du coût $J$.

**Instructions**:
- Il y a plusieurs façon d'implémenter la loss cross-entropy. Pour vous aider, on vous met ici comment on aurait calculé $- \sum\limits_{i=0}^{m}  y^{(i)}\log(a^{[2](i)})$:
```python
logprobs = np.multiply(np.log(A2),Y)
cost = - np.sum(logprobs)                # pas besoin d'utiliser une boucle for !
```

(vous pouvez utiliser `np.multiply()` puis `np.sum()` ou directement `np.dot()`).

In [ ]:
def compute_cost(A2, Y, parameters):
    """
    Calculez le coût cross-entropy dont la formule se trouve dans l'équation (13)
    
    Arguments:
    A2 -- L'output de la sigmoïde de la deuxième activation, de shape (1, nombre d'exemples)
    Y -- vecteur contenant les labels, de shape (1, number of examples)
    parameters -- dictionnaire python contenant vos paramètres W1, b1, W2 and b2
    
    Return:
    cost -- coût cross-entropy selon l'équation (13)
    """
    m = Y.shape[1] # nombre d'exemples

    # Calculez la cross entropy
    ### Début du code ### (≈ 2 lignes de code)
    logprobs = None
    cost = None
    ### Fin du code ###
    
    cost = np.squeeze(cost)     
    assert(isinstance(cost, float))
    
    return cost

In [ ]:
A2, Y_assess, parameters = compute_cost_test_case()

print("cost = " + str(compute_cost(A2, Y_assess, parameters)))

**Résultat attendu**:
<table style="width:20%">
  <tr>
    <td>cost</td>
    <td> 0.693058761... </td> 
  </tr>
  
</table>

Grâce au dictionnaire "cache" que vous avez calculé pendant la forward propagation, vous pouvez maintenant implémenter la backward propagation.

**Exercice**: Implementez la fonction `backward_propagation()`.

**Instructions**:
La backpropagation est généralement la partie la plus compliquée en deep learning (la plus mathématique). Pour vous aider, on vous a mis ici les différentes étapes de la backpropagation.  

<img src="images/grad_summary.png" style="width:600px;height:300px;">

<!--
$\frac{\partial \mathcal{J} }{ \partial z_{2}^{(i)} } = \frac{1}{m} (a^{[2](i)} - y^{(i)})$

$\frac{\partial \mathcal{J} }{ \partial W_2 } = \frac{\partial \mathcal{J} }{ \partial z_{2}^{(i)} } a^{[1] (i) T} $

$\frac{\partial \mathcal{J} }{ \partial b_2 } = \sum_i{\frac{\partial \mathcal{J} }{ \partial z_{2}^{(i)}}}$

$\frac{\partial \mathcal{J} }{ \partial z_{1}^{(i)} } =  W_2^T \frac{\partial \mathcal{J} }{ \partial z_{2}^{(i)} } * ( 1 - a^{[1] (i) 2}) $

$\frac{\partial \mathcal{J} }{ \partial W_1 } = \frac{\partial \mathcal{J} }{ \partial z_{1}^{(i)} }  X^T $

$\frac{\partial \mathcal{J} _i }{ \partial b_1 } = \sum_i{\frac{\partial \mathcal{J} }{ \partial z_{1}^{(i)}}}$

- Note that $*$ denotes elementwise multiplication.
- The notation you will use is common in deep learning coding:
    - dW1 = $\frac{\partial \mathcal{J} }{ \partial W_1 }$
    - db1 = $\frac{\partial \mathcal{J} }{ \partial b_1 }$
    - dW2 = $\frac{\partial \mathcal{J} }{ \partial W_2 }$
    - db2 = $\frac{\partial \mathcal{J} }{ \partial b_2 }$
    
!-->

- Indice:
    - Pour calculer dZ1 vous devrez calculer $g^{[1]'}(Z^{[1]})$. Comme $g^{[1]}(.)$ est la fonction d'activation tanh, si $a = g^{[1]}(z)$ alors $g^{[1]'}(z) = 1-a^2$. Vous pourrez ensuite calculer 
    $g^{[1]'}(Z^{[1]})$ avec `(1 - np.power(A1, 2))`.

In [ ]:
def backward_propagation(parameters, cache, X, Y):
    """
    Implémentez la backward propagation en suivant les instructions ci-dessus
    
    Arguments:
    parameters -- dictionnaire python contenant vos paramètres
    cache -- dictionnaire python contenant "Z1", "A1", "Z2"  et "A2".
    X -- données en entrée de shape (2, nombre d'exemples)
    Y -- vecteur des labels de shape (1, nombre d'exemples)
    
    Return:
    grads -- dictionnaire python contenant vos gradients
    """
    m = X.shape[1]
    
    # Récuperez W1 et W2 à partie de parameters
    ### Début du code ### (≈ 2 lignes de code)
    W1 = None
    W2 = None
    ### Fin du code ###
        
    # Récupérez également A1 et A2 depuis le dictionnaire "cache".
    ### Début du code ### (≈ 2 lignes de code)
    A1 = None
    A2 = None
    ### Fin du code ###
    
    # Backward propagation: calculez les gradients dW1, db1, dW2, db2. 
    ### Début du code ### (≈ 6 lignes de code, correspondant aux 6 équations de la cellule au-dessus)
    dZ2 = None
    dW2 = None
    db2 = None
    dZ1 = None
    dW1 = None
    db1 = None
    ### Fin du code ###
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

In [ ]:
parameters, cache, X_assess, Y_assess = backward_propagation_test_case()

grads = backward_propagation(parameters, cache, X_assess, Y_assess)
print ("dW1 = "+ str(grads["dW1"]))
print ("db1 = "+ str(grads["db1"]))
print ("dW2 = "+ str(grads["dW2"]))
print ("db2 = "+ str(grads["db2"]))

**Résultat attendu**:



<table style="width:80%">
  <tr>
    <td>dW1</td>
    <td> [[ 0.00301023 -0.00747267]
 [ 0.00257968 -0.00641288]
 [-0.00156892  0.003893  ]
 [-0.00652037  0.01618243]] </td> 
  </tr>
  
  <tr>
    <td>db1</td>
    <td>  [[ 0.00176201]
 [ 0.00150995]
 [-0.00091736]
 [-0.00381422]] </td> 
  </tr>
  
  <tr>
    <td>dW2</td>
    <td> [[ 0.00078841  0.01765429 -0.00084166 -0.01022527]] </td> 
  </tr>
  

  <tr>
    <td>db2</td>
    <td> [[-0.16655712]] </td> 
  </tr>
  
</table>  

**Exercice**: Implementez la fonction update_parameters. Utilisez la descente de gradient. Vous devez vous servir de (dW1, db1, dW2, db2) afin d'update (W1, b1, W2, b2).

**Règle générale de la descente de gradient**: $ \theta = \theta - \alpha \frac{\partial J }{ \partial \theta }$ avec $\alpha$ le learning rate et $\theta$ représentant le paramètre.

**Illustration**: Vous pouvez comparer ci-dessous la différence avec un bon learning rate (convergence) et un mauvais learning rate (divergence). Crédits animation: Adam Harley.

<img src="images/sgd.gif" style="width:400;height:400;"> <img src="images/sgd_bad.gif" style="width:400;height:400;">



In [ ]:
def update_parameters(parameters, grads, lr = 1.2):
    """
    Arguments:
    parameters -- dictionnaire python contenant vos paramètres
    grads -- dictionnaire python contenant vos gradients 
    
    Return:
    parameters -- dictionnaire python contenant vos paramètres updatés
    """
    # Récuperer vos paramètres
    ### Début du code ### (≈ 4 lignes de code)
    W1 = None
    b1 = None
    W2 = None
    b2 = None
    ### Fin du code ###
    
    # Récupérez vos gradients
    ### Début du code ### (≈ 4 lignes de code)
    dW1 = None
    db1 = None
    dW2 = None
    db2 = None
    ## Fin du code ###
    
    # Updatez vos paramètres 
    ### Début du code ### (≈ 4 lignes de code)
    W1 = None
    b1 = None
    W2 = None
    b2 = None
    ### Fin du code ###
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [ ]:
parameters, grads = update_parameters_test_case()
parameters = update_parameters(parameters, grads)

print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

**Résultat attendu**:


<table style="width:80%">
  <tr>
    <td>W1</td>
    <td> [[-0.00643025  0.01936718]
 [-0.02410458  0.03978052]
 [-0.01653973 -0.02096177]
 [ 0.01046864 -0.05990141]]</td> 
  </tr>
  
  <tr>
    <td>b1</td>
    <td> [[ -1.02420756e-06]
 [  1.27373948e-05]
 [  8.32996807e-07]
 [ -3.20136836e-06]]</td> 
  </tr>
  
  <tr>
    <td>W2</td>
    <td> [[-0.01041081 -0.04463285  0.01758031  0.04747113]] </td> 
  </tr>
  

  <tr>
    <td>b2</td>
    <td> [[ 0.00010457]] </td> 
  </tr>
  
</table>  

### 4.4 - Intégrez les parties 4.1, 4.2 et 4.3 dans nn_model() ####

**Exercice**: Construisez votre réseau de neurones dans `nn_model()`.

**Instructions**: Le réseau de neurones doit utiliser vos fonctions précédentes dans le bon ordre.

In [ ]:
def nn_model(X, Y, n_h, num_iterations = 10000, print_cost=False):
    """
    Arguments:
    X -- dataset de shape (2, nombre d'exemples)
    Y -- labels de shape (1, nombre d'exemples)
    n_h -- size de la couche cachée
    num_iterations -- Nombre d'itérations dans la boucle 
    print_cost -- si True, print le coût toutes les 1000 itérations
    
    Return:
    parameters -- paramètres entraînés. Ce sont ces paramètres qui seront utilisés pour de nouvelles données.
    """
    
    np.random.seed(3)
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[2]
    
    # Initialise les paramètres, puis récupère W1, b1, W2, b2. Inputs: "n_x, n_h, n_y". Outputs = "W1, b1, W2, b2, parameters".
    ### Début du code ### (≈ 5 lignes de code)
    parameters = None
    W1 = None
    b1 = None
    W2 = None
    b2 = None]
    ### Fin du code ###
    
    # Boucle

    for i in range(0, num_iterations):
         
        ### Début du code ### (≈ 4 lignes de code)
        # Forward propagation. Inputs: "X, parameters". Outputs: "A2, cache".
        A2, cache = None
        
        # Fonction de coût. Inputs: "A2, Y, parameters". Outputs: "cost".
        cost = None
 
        # Backpropagation. Inputs: "parameters, cache, X, Y". Outputs: "grads".
        grads = None
 
        # Update des paramètres. Inputs: "parameters, grads". Outputs: "parameters".
        parameters = None
        
        ### Fin du code ###
        
        # Print le coût toutes les 1000 iterations
        if print_cost and i % 1000 == 0:
            print ("Coût après itération %i: %f" %(i, cost))

    return parameters

In [ ]:
X_assess, Y_assess = nn_model_test_case()
parameters = nn_model(X_assess, Y_assess, 4, num_iterations=10000, print_cost=True)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

**Résultat attendu**:

<table style="width:90%">

<tr> 
    <td> 
        coût après itération 0
    </td>
    <td> 
        0.693207
    </td>
</tr>

<tr> 
    <td> 
        <center> $\vdots$ </center>
    </td>
    <td> 
        <center> $\vdots$ </center>
    </td>
</tr>

  <tr>
    <td>W1</td>
    <td> [[-0.60314397  1.127818  ]
 [-0.74257953  1.3776121 ]
 [-0.69186287  1.27565127]
 [-0.7349128   1.36877347]]</td> 
  </tr>
  
  <tr>
    <td>b1</td>
    <td> [[0.26051203]
 [0.35648363]
 [0.31850012]
 [0.35292858]] </td> 
  </tr>
  
  <tr>
    <td>W2</td>
    <td> [[-2.10582454 -3.16994587 -2.70047639 -3.11996342]] </td> 
  </tr>
  

  <tr>
    <td>b2</td>
    <td> [[0.21989657]] </td> 
  </tr>
  
</table>  

### 4.5 Prédictions

**Exercice**: Utilisez votre modèle pour prédire de nouvelles données. Implémentez predict().
Utilisez la forward propagation

**Rappel**: predictions = $y_{prediction} = \mathbb 1 \text{{activation > 0.5}} = \begin{cases}
      1 & \text{si}\ activation > 0.5 \\
      0 & \text{sinon}
    \end{cases}$  
    
Par exemple, si vous voulez définir les éléments d'une matrice X à 0 ou 1 en fonction d'un seuil, vous pouvez faire: ```X_new = (X > threshold)```

In [ ]:
def predict(parameters, X):
    """
    Grâce aux paramètres appris, prédisez la classe pour chaque exemple de X
    
    Arguments:
    parameters -- dictionnaire python contenant vos paramètres
    X -- données en entrée de siwe (n_x, m)
    
    Returns
    predictions -- vecteur des prédictions (rouge: 0 / bleu: 1)
    """
    
    # Calcule les propabilités en utilisant la forward propagation, et classifie en 0 ou 1 grâce au seuil défini comme 0.5
    ### Début du code ### (≈ 2 lignes de code)
    A, cache = None
    predictions = None
    ### Fin du code ###
    
    return predictions

In [ ]:
parameters, X_assess = predict_test_case()

predictions = predict(parameters, X_assess)
print("moyenne des predictions = " + str(np.mean(predictions)))

**Résultat attendu**: 


<table style="width:40%">
  <tr>
    <td>moyenne des predictions</td>
    <td> 0.666666666667 </td> 
  </tr>
  
</table>

On peut maintenant entraîner notre modèle sur notre jeu de données. Exécutez le code suivant pour tester votre modèle avec une couche cachée de $n_h$ neurones. 

In [ ]:
n_h = 5
parameters = nn_model(X, Y, n_h = n_h, num_iterations = 10000, print_cost=True)

# Plot la decision boundary
try:
    plot_decision_boundary(lambda x: predict(parameters, x.T), X, Y)
except:
    print("Suite à un problème de version matplotlib, vous ne voyez pas les points initiaux. Reprenez le graphique au début du notebook et imaginez les points comme étant plot.")
    pass
plt.title("Decision Boundary pour une couche cachée de size " + str(n_h))

**Résultat attendu**:

<table style="width:40%">
  <tr>
    <td>Coût après itération 9000</td>
    <td> 0.178517 </td> 
  </tr>
  
</table>


In [ ]:
# Print l'accurary
predictions = predict(parameters, X)
print ('Accuracy: %d' % float((np.dot(Y,predictions.T) + np.dot(1-Y,1-predictions.T))/float(Y.size)*100) + '%')

**Résultat attendu**: 

<table style="width:15%">
  <tr>
    <td>Accuracy</td>
    <td> 91% </td> 
  </tr>
</table>

L'accurary est beaucoup plus élevé que celui de la régression logistique. Le modèle a pu apprendre le pattern des feuilles de la fleur. Contrairement à la régression logistique, les réseaux de neurones profonds sont capables de traiter des problèmes non linéaires.

Essayez maintenant avec plusieurs nombres de neurones différents dans la couche cachée. 

### 4.6 - Tuning du nombre de neurones dans la couche cachée ###

Exécutez le code suivant. Cela devrait prendre 1-2 minutes. Vous observerez des comportements différents du modèle selon le nombre de neurones que vous aurez défini dans la couche cachée.

In [ ]:
# Cela devrait prendre 2 minutes

plt.figure(figsize=(16, 32))
hidden_layer_sizes = [1, 2, 3, 4, 5, 20, 50]
for i, n_h in enumerate(hidden_layer_sizes):
    plt.subplot(5, 2, i+1)
    plt.title('Hidden Layer of size %d' % n_h)
    parameters = nn_model(X, Y, n_h, num_iterations = 5000)
    try:
        plot_decision_boundary(lambda x: predict(parameters, x.T), X, Y)
    except:
        print("Suite à un problème de version matplotlib, vous ne voyez pas les points initiaux. Reprenez le graphique au début du notebook et imaginez les points comme étant plot.")
    predictions = predict(parameters, X)
    accuracy = float((np.dot(Y,predictions.T) + np.dot(1-Y,1-predictions.T))/float(Y.size)*100)
    print ("Accuracy for {} hidden units: {} %".format(n_h, accuracy))

**Interpretation**:
- Le plus gros modèle est capable de mieux classifier le jeu de données de train, jusqu'à overfitter les données. 
- Le meilleur modèle semble être celui avec 5 neurones.

Une méthode pour éviter l'overfitting est la régularisation. Nous aurons peut etre l'occasion de voir ça dans un futur workshop.

References:
- http://scs.ryerson.ca/~aharley/neural-networks/
- http://cs231n.github.io/neural-networks-case-study/